In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/voz-comments/comments_data.csv


In [2]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# comments_full=pd.read_csv('/kaggle/input/voz-comments/comments_data.xlsx - Sheet1 (1).csv', usecols=['Emotion', 'Sentence'])

# Modify 'score' based on your conditions using vectorized operations
#comments_full['score'] = (comments_full['score'] >= 1).astype(int)
# Sampling 40,000 random records from the DataFrame
#comments = comments_full.sample(n=1000, random_state=42)

# Print the first few rows of the sampled DataFrame
comments=pd.read_csv('/kaggle/input/voz-comments/comments_data.csv', usecols=['Emotion', 'Sentence'])
comments = comments.dropna(subset=['Emotion'])
# Assuming 'comments' is your DataFrame and it's already cleaned of NaNs in 'Emotion'
comments['Emotion'] = comments['Emotion'].astype(int)  # Ensure Emotion is of integer type
# comments = comments.sample(n=100, random_state=42)
print(comments.head())


print(f"Số lượng comments: {comments.shape[0]}")
print(f"comments {type(comments['Sentence'].tolist())}")
unique_emotions = comments['Emotion'].unique()
print(unique_emotions)



   Emotion                                           Sentence
0        1  Hình ảnh setup menu ko có gì đặc biệt chỉ có n...
1        0                      vote tậu thêm arm t9 pro a ơi
2        1                       giữ chỗ tối update chi tiết.
3        1  Góc bàn của thằng bạn e. Nhìn màn hình như lơ ...
4        1  Của mình k có chỗ gắn do bàn để lơ lửng k có c...
Số lượng comments: 4954
comments <class 'list'>
[1 0]


In [3]:
!pip install transformers torch


In [4]:

import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Make sure you've also installed and imported pandas if you're working with DataFrame
import pandas as pd
#model_name = "vinai/phobert-base"
model_name="vinai/phobert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Assuming binary classification (positive/negative)
num_labels = 2
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
# Ensure all entries are strings (this also handles NaN and other non-string types)
comments['Sentence'] = comments['Sentence'].astype(str)
encoded_comments = tokenizer.batch_encode_plus(
    comments['Sentence'].tolist(),  # The comments
    add_special_tokens=True,  # Add '[CLS]' and '[SEP]'
    padding='max_length',  # Pad to max_length
    max_length=256,  # Adjust based on your needs
    truncation=True,  # Truncate to max_length
    return_attention_mask=True,  # Include attention masks
    return_tensors='pt'  # Return PyTorch tensors
)

input_ids = encoded_comments['input_ids']
attention_masks = encoded_comments['attention_mask']
labels = torch.tensor(comments['Emotion'].values)


print(f"attention_masks{attention_masks}")
print(f"labels{labels}")


attention_maskstensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
labelstensor([1, 0, 1,  ..., 1, 1, 0])


In [6]:
# Create the TensorDataset
dataset = TensorDataset(input_ids, attention_masks, labels)

# Split into training and validation sets
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create DataLoaders
batch_size = 16  # Adjust based on your GPU memory

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
print(val_loader)

In [7]:
from tqdm import tqdm  # Import the tqdm library
from torch.optim import AdamW
# Prepare optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Training loop
model.train()
num_epochs=1
for epoch in range(num_epochs):
    # Wrap your loader with tqdm for a progress bar
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")
    
    for batch in progress_bar:
        # Unpack batch and move to the same device as model
        input_ids, attention_mask, labels = batch
        input_ids = input_ids.to(model.device)
        attention_mask = attention_mask.to(model.device)
        labels = labels.to(model.device)

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        
        # Compute loss and backpropagate
        loss = outputs.loss
        loss.backward()
        
        # Update parameters and clear gradients
        optimizer.step()
        optimizer.zero_grad()
        
        # Update the progress bar with the current loss
        progress_bar.set_postfix({'loss': loss.item()})


Epoch 1/1: 100%|██████████| 279/279 [1:30:55<00:00, 19.55s/it, loss=0.705]


In [8]:
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch

model.eval()  # Set the model to evaluation mode

true_labels = []
predictions = []

with torch.no_grad():  # No need to track gradients
    for batch in tqdm(val_loader, desc="Evaluating"):
        input_ids, attention_mask, labels = batch
        input_ids = input_ids.to(model.device)
        attention_mask = attention_mask.to(model.device)
        labels = labels.to(model.device)
        
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        
        # Convert logits to probabilities using softmax
        probabilities = torch.softmax(logits, dim=1).cpu().numpy()  # Convert to numpy array immediately
        
        # Apply the custom threshold to decide between classes
        threshold = 0.3  # Adjust based on your observations
        pred_labels_custom_threshold = (probabilities[:, 1] > threshold).astype(int)  # Use numpy operation
        
        # Append predictions and true labels
        predictions.extend(pred_labels_custom_threshold)  # It's already a numpy array
        true_labels.extend(labels.cpu().numpy())

# Calculate evaluation metrics
accuracy = accuracy_score(true_labels, predictions)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='binary')


# Example: Assuming `logits` is a tensor from your model's output
probabilities = torch.softmax(logits, dim=1).cpu().numpy()
print("Sample probabilities:", probabilities[:5])  # Print first 5 samples' probabilities

# Check if predictions are skewed towards one class
unique, counts = np.unique(predictions, return_counts=True)
print("Prediction counts:", dict(zip(unique, counts)))

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


Evaluating: 100%|██████████| 31/31 [03:00<00:00,  5.83s/it]

Sample probabilities: [[0.4215067  0.5784933 ]
 [0.6496027  0.3503973 ]
 [0.6495529  0.35044715]
 [0.4156814  0.5843186 ]
 [0.6530123  0.34698772]]
Prediction counts: {0: 38, 1: 458}
Accuracy: 0.4476
Precision: 0.4258
Recall: 0.9466
F1 Score: 0.5873
